In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
df = pd.read_json("/home/zeng/Desktop/H3N2/data/Isolation_information/ioslation_information.json", orient='index')

In [10]:
df['region'] = df['location'].str.split(' / ').str.get(0)
df['country'] = df['location'].str.split(' / ').str.get(1)
df['datetime'] = pd.to_datetime(df['date'])

In [14]:
df.tail()

,HA,MP,NA,NP,NS,PA,PB1,PB2,date,location,name,region,country,datetime
EPI_ISL_376421,EPI1538401,EPI1538428,EPI1538455,EPI1538482,EPI1538509,EPI1538536,EPI1538552,EPI1538576,2018-10-31,North America / United States / Texas,A/Texas/7538/2018,North America,United States,2018-10-31
EPI_ISL_376412,EPI1538392,EPI1538419,EPI1538446,EPI1538473,EPI1538500,EPI1538527,EPI1538548,EPI1538568,2018-10-30,North America / United States / New Jersey,A/New Jersey/7529/2018,North America,United States,2018-10-30
EPI_ISL_376411,EPI1538391,EPI1538418,EPI1538445,EPI1538472,EPI1538499,EPI1538526,EPI1538547,EPI1538567,2018-11-02,North America / United States / New Jersey,A/New Jersey/7528/2018,North America,United States,2018-11-02
EPI_ISL_376407,EPI1538387,EPI1538414,EPI1538441,EPI1538468,EPI1538495,EPI1538522,EPI1538544,EPI1538564,2018-10-22,North America / United States / Hawaii,A/Hawaii/7524/2018,North America,United States,2018-10-22
EPI_ISL_376399,EPI1538379,EPI1538406,EPI1538433,EPI1538460,EPI1538487,EPI1538514,EPI1538541,EPI1538557,2018-10-30,North America / United States / Alaska,A/Alaska/7516/2018,North America,United States,2018-10-30


In [12]:
df['region'].value_counts()

North America    32651
Asia             22127
Europe           19666
Oceania           6581
South America     4083
Africa            2516
/ Taiwan             5
/ Laos               4
/ SPAIN              4
                     1
Name: region, dtype: int64

In [16]:
df_us = df[df['country'] == 'United States']

In [3]:
df = pd.read_json("/home/zeng/Desktop/H3N2/data/Isolation_information/ioslation_information.json", orient='index')
df['region'] = df['location'].str.split(' / ').str.get(0)
df['country'] = df['location'].str.split(' / ').str.get(1)
df['datetime'] = pd.to_datetime(df['date'])

df_us = df[df['country'] == 'United States']

In [68]:
df_us[df_us['HA'].notna()].head()

,HA,MP,NA,NP,NS,PA,PB1,PB2,date,location,name,region,country,datetime
EPI_ISL_188281,EPI609755,,,,,,,,2001-11-22,North America / United States / New York,A/New York/55/2001,North America,United States,2001-11-22
EPI_ISL_167298,EPI545337,,,,,,,,2004-01-01,North America / United States,A/New York/55/2004,North America,United States,2004-01-01
EPI_ISL_154544,EPI502273,EPI502275,EPI502277,EPI502279,EPI502281,EPI501853,EPI502283,EPI502285,2005-01-01,North America / United States / Wisconsin,A/Wisconsin/67/2005,North America,United States,2005-01-01
EPI_ISL_154543,EPI502219,EPI502220,EPI502221,EPI502222,EPI502223,EPI501852,EPI502224,EPI502225,2005-01-01,North America / United States / Washington,A/Wisconsin/67/2005,North America,United States,2005-01-01
EPI_ISL_154540,EPI502300,EPI502302,EPI502304,EPI502306,EPI502308,EPI501849,EPI502310,EPI502312,2005-01-01,North America / United States / Wisconsin,A/Wisconsin/67/2005,North America,United States,2005-01-01


In [73]:
epi[:10]

['EPI_ISL_188281',
 'EPI_ISL_167298',
 'EPI_ISL_154544',
 'EPI_ISL_154543',
 'EPI_ISL_154540',
 'EPI_ISL_154534',
 'EPI_ISL_154522',
 'EPI_ISL_132393',
 'EPI_ISL_114293',
 'EPI_ISL_109778']

In [74]:
ha[:10]

['EPI609755',
 'EPI545337',
 'EPI502273',
 'EPI502219',
 'EPI502300',
 'EPI502240',
 'EPI502156',
 'EPI406672',
 'EPI365578',
 'EPI360972']

In [1]:
import json

with open("/home/zeng/Desktop/H3N2/data/Isolation_information/ioslation_information.json") as f:
    contents = f.read()

json_file = json.loads(contents)

In [7]:
dict(json_file)["EPI_ISL_124118"]

{'PB2': '',
 'PB1': '',
 'PA': '',
 'HA': 'EPI383634',
 'NP': '',
 'NA': '',
 'MP': '',
 'NS': '',
 'date': '2005-02',
 'name': 'A/Pavia/3063/2005',
 'location': 'Europe / Italy'}

In [71]:
value = 'HA'
df_us_notno = df_us[df_us[value].notna()]
epi = df_us_notno.index.to_list()
ha = df_us_notno[value].to_list()
dt = [x.strftime("%Y-%m-%d") for x in df_us_notno.datetime.to_list()]